## SMAVA

Test exercise - Question 1

In [1]:
## weaves
## smava

rm(list=ls())

getwd()

## load in packages
library(Hmisc)

library(ranger)
library(MASS)
library(tidyverse)
library(e1071)

library(rpart)
library(rpart.plot)
library(ipred)
library(mlbench)
library(pROC)
library(gbm)
library(dplyr)
library(caret)

library(doMC)

registerDoMC(cores = detectCores(all.tests = FALSE, logical = TRUE))

options(useFancyQuotes = TRUE)

[1] "/misc/build/0/pyeg0/caret/classr1"

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: 'Hmisc'


The following objects are masked from 'package:base':

    format.pval, units


-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v tibble  2.1.3     v dplyr   0.8.3
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0
v purrr   0.3.3     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter()    masks stats::filter()
x dplyr::lag()       masks stats::lag()
x dplyr::select()    masks MASS::select()
x dplyr::src()       masks Hmisc::src()
x dplyr::summarize() masks Hmisc::summarize()


Attaching package: 'e1071'


The following object is masked from 'package:Hmisc':

    impute



Attaching package: 'rpart'


The following object is masked from 'package:survival':

    solder


Type 'citation("pROC")' for a citation.


Attac

In [2]:
## Data sets
load("smava00.dat")
load("bak/in/test.rdata")
ls()

[1] "outcomes" "smava0"   "test"     "train1"

In [3]:
xcols <- setdiff(smava0$ft0, c(smava0$hcor, "outcomes"))
xcols

[1] "x1"       "x2"       "x4"       "x5"       "x6"       "x7"      
 [7] "x8"       "x9"       "x10"      "x2na"     "accepted" "bank"

In [16]:
## Rename/Preprocess
df0 <- train1[, xcols]

# Null check.\
# sapply(df0, summary, USE.NAMES=TRUE)

head(df0)

x1,x2,x4,x5,x6,x7,x8,x9,x10,x2na,accepted,bank
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
0.9954692,3.688772,2.332774,3.033258,1,803,5.516418,0.5990008,0.406155,0,YES,6
0.9954692,3.688772,2.332774,3.033258,1,803,5.412266,0.5949548,0.406155,0,YES,9
0.9954692,3.688772,2.332774,3.033258,1,803,5.325315,0.6043728,0.406155,0,NO,15
0.9954692,3.688772,2.332774,3.033258,1,803,5.212415,0.5872142,0.406155,0,NO,21
0.9954692,3.688772,2.332774,3.033258,1,803,5.022601,0.6331760,0.406155,0,YES,22
0.9954692,3.688772,2.332774,3.033258,1,803,5.172827,0.6448435,0.406155,0,YES,24


In [31]:
## simple controls

fitControl <- trainControl(
    method = "repeatedcv",
    number = 5, ## repeated a few times
    repeats = 3,
    summaryFunction = twoClassSummary,
  classProbs = TRUE)

gbmGrid <- expand.grid(interaction.depth = c(1, 5, 7), # 7 gives a marginal improvement.
                       n.trees = (10:20)*10,
                       shrinkage = 0.1,
                       n.minobsinnode = 20)

# methods: parRF and ranger crash
method0 <- "gbm"; metric0 <- "ROC"

In [32]:
fit0 <- train(accepted ~ ., data = df0, method = method0, 
              trControl = fitControl,
              tuneGrid = gbmGrid,
              metric = metric0,
              verbose = FALSE)
smava0$gbm <- fit0
fit0

Stochastic Gradient Boosting 

134969 samples
    11 predictor
     2 classes: 'NO', 'YES' 

No pre-processing
Resampling: Cross-Validated (5 fold, repeated 3 times) 
Summary of sample sizes: 107975, 107975, 107975, 107976, 107975, 107975, ... 
Resampling results across tuning parameters:

  interaction.depth  n.trees  ROC        Sens       Spec     
  1                  100      0.6602236  0.6448472  0.5966069
  1                  110      0.6622285  0.6499084  0.5943948
  1                  120      0.6635030  0.6520977  0.5943648
  1                  130      0.6648415  0.6542479  0.5936591
  1                  140      0.6659374  0.6553498  0.5939244
  1                  150      0.6670003  0.6546476  0.5962315
  1                  160      0.6679539  0.6523658  0.6003753
  1                  170      0.6687132  0.6553742  0.5986037
  1                  180      0.6695693  0.6559057  0.6004004
  1                  190      0.6702690  0.6579096  0.5995896
  1                  200   

In [33]:
trainPred <- predict(fit0, df0)
# postResample(testPred, testClass)

conf0 <- confusionMatrix(trainPred, df0$accepted, positive = "YES")
conf0

## Only 65% accurate on the whole set!

nvars <- floor(length(colnames(df0)) * 2/3)

jpeg(filename=paste("smava0", "mf", "-%03d.jpeg", sep=""), 
     width=1024, height=768)

modelImp <- varImp(fit0, scale = FALSE)
plot(modelImp, top = min(dim(modelImp$importance)[1], nvars) )

## Get a density and a ROC
## You need twoClassSummary for this

x.p <- predict(fit0, df0, type = "prob")[2]

test.df <- data.frame(true0=x.p[[ "YES" ]], Obs=outcomes)
test.roc <- roc(Obs ~ true0, test.df)

densityplot(~test.df$true0, groups = test.df$Obs, auto.key = TRUE)

plot.roc(test.roc)

dev.off()

Confusion Matrix and Statistics

          Reference
Prediction    NO   YES
       NO  49031 26567
       YES 19333 40038
                                          
               Accuracy : 0.6599          
                 95% CI : (0.6574, 0.6624)
    No Information Rate : 0.5065          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.3188          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.6011          
            Specificity : 0.7172          
         Pos Pred Value : 0.6744          
         Neg Pred Value : 0.6486          
             Prevalence : 0.4935          
         Detection Rate : 0.2966          
   Detection Prevalence : 0.4399          
      Balanced Accuracy : 0.6592          
                                          
       'Positive' Class : YES             
                                          

Setting levels: control = 0, case = 1

Setting direction: controls < cases



png 
  2

In [8]:
## Make a prediction using test.

## Apply the same data pre-processing and predict.

## Don't do the order.

test0 <- data.frame(test) # just an backup to use in the interpreter.
test1 <- data.frame(test)

for(c in intersect(smava0$cs, colnames(test))) {
  test1[[c]] <- as.numeric(test1[[c]])
}

In [9]:
test1$x2na <- 0
test1[ is.na(test1$x2), "x2na" ] <- 1
test1[ is.na(test1$x2), "x2" ] <- smava0$x2impute

In [10]:
df0 <- test1[, intersect(cols, colnames(test1))]

In [11]:
## pre-process
if (!is.null(smava0$pp)) {
    df0 <- predict(smava0$pp, df0)
}

testPred <- predict(smava0$gbm, df0)

predictions <- data.frame(test)
predictions$predictionAccepted <- testPred

save(predictions, file="smava01.dat")

In [12]:
head(predictions)

,customerNumber,bank,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,predictionAccepted
,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>
1,17652,B4,2.6999043,NA,289.8060,3.5062855,3.019291,ACS,P0DI0,6.132490,0.2024692,0.2340533,NO
2,5083,B2,1.7768174,NA,149.4808,8.6785405,1.561671,ACS,WXATN,9.602027,0.6035512,0.4728191,NO
3,14485,B27,9.2014608,15.2018994,168.5305,0.6861547,-2.659850,FBW,YVNYM,-1.843895,0.3604684,0.4297722,NO
4,24278,B12,0.2425088,0.8906478,144.3519,8.9942867,3.757424,ACS,WXATN,13.329596,0.2102926,0.4851283,NO
5,18816,B29,1.8619350,0.9441711,330.2783,11.1829245,1.795021,FBW,KT4RM,12.944497,0.1193745,0.1910898,YES
6,25518,B25,8.9591239,9.3258122,138.9361,9.5992959,4.778473,ACS,LURGJ,14.795508,0.9447004,0.4984742,YES
